In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [8]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(2e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(2e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(3e-4)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(3e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.45))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [9]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
3095/3095 - 151s - loss: 0.5271 - accuracy: 0.8240 - val_loss: 0.6606 - val_accuracy: 0.7991
Epoch 2/10
3095/3095 - 149s - loss: 0.3573 - accuracy: 0.8815 - val_loss: 1.0140 - val_accuracy: 0.7422
Epoch 3/10
3095/3095 - 149s - loss: 0.2966 - accuracy: 0.9046 - val_loss: 0.9322 - val_accuracy: 0.7402
Epoch 4/10
3095/3095 - 149s - loss: 0.2672 - accuracy: 0.9135 - val_loss: 0.5167 - val_accuracy: 0.8246
Epoch 5/10
3095/3095 - 149s - loss: 0.2539 - accuracy: 0.9194 - val_loss: 0.7773 - val_accuracy: 0.7965
Epoch 6/10
3095/3095 - 149s - loss: 0.2476 - accuracy: 0.9218 - val_loss: 0.4299 - val_accuracy: 0.8376
Epoch 7/10
3095/3095 - 149s - loss: 0.2422 - accuracy: 0.9234 - val_loss: 0.3845 - val_accuracy: 0.9076
Epoch 8/10
3095/3095 - 149s - loss: 0.2367 - accuracy: 0.9266 - val_loss: 0.2301 - val_accuracy: 0.9273
Epoch 9/10
3095/3095 - 149s - loss: 0.2338 - accuracy: 0.9269 - val_loss: 0.2402 - val_accuracy: 0.9266
Epoch 10/10
3095/3095 - 149s - loss: 0.2314 - accuracy: 0.9281 -

In [10]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
1548/1548 - 160s - loss: 0.1855 - accuracy: 0.9460 - val_loss: 0.1690 - val_accuracy: 0.9521
Epoch 2/10
1548/1548 - 157s - loss: 0.1752 - accuracy: 0.9486 - val_loss: 0.1939 - val_accuracy: 0.9395
Epoch 3/10
1548/1548 - 157s - loss: 0.1685 - accuracy: 0.9507 - val_loss: 0.1666 - val_accuracy: 0.9499
Epoch 4/10
1548/1548 - 159s - loss: 0.1642 - accuracy: 0.9515 - val_loss: 0.1545 - val_accuracy: 0.9531
Epoch 5/10
1548/1548 - 157s - loss: 0.1600 - accuracy: 0.9526 - val_loss: 0.2099 - val_accuracy: 0.9316
Epoch 6/10
1548/1548 - 148s - loss: 0.1574 - accuracy: 0.9535 - val_loss: 0.1585 - val_accuracy: 0.9522
Epoch 7/10
1548/1548 - 149s - loss: 0.1545 - accuracy: 0.9541 - val_loss: 0.1652 - val_accuracy: 0.9503
Epoch 8/10
1548/1548 - 148s - loss: 0.1530 - accuracy: 0.9541 - val_loss: 0.1449 - val_accuracy: 0.9578
Epoch 9/10
1548/1548 - 148s - loss: 0.1507 - accuracy: 0.9549 - val_loss: 0.1739 - val_accuracy: 0.9434
Epoch 10/10
1548/1548 - 148s - loss: 0.1487 - accuracy: 0.9560 -

In [11]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 164s - loss: 0.1339 - accuracy: 0.9612 - val_loss: 0.1368 - val_accuracy: 0.9593
Epoch 2/10
774/774 - 149s - loss: 0.1317 - accuracy: 0.9618 - val_loss: 0.1419 - val_accuracy: 0.9573
Epoch 3/10
774/774 - 158s - loss: 0.1299 - accuracy: 0.9624 - val_loss: 0.1447 - val_accuracy: 0.9561
Epoch 4/10
774/774 - 159s - loss: 0.1308 - accuracy: 0.9619 - val_loss: 0.1361 - val_accuracy: 0.9596
Epoch 5/10
774/774 - 160s - loss: 0.1282 - accuracy: 0.9628 - val_loss: 0.1387 - val_accuracy: 0.9590
Epoch 6/10
774/774 - 159s - loss: 0.1277 - accuracy: 0.9631 - val_loss: 0.1348 - val_accuracy: 0.9607
Epoch 7/10
774/774 - 161s - loss: 0.1263 - accuracy: 0.9637 - val_loss: 0.1375 - val_accuracy: 0.9597
Epoch 8/10
774/774 - 161s - loss: 0.1260 - accuracy: 0.9636 - val_loss: 0.1327 - val_accuracy: 0.9613
Epoch 9/10
774/774 - 161s - loss: 0.1255 - accuracy: 0.9637 - val_loss: 0.1331 - val_accuracy: 0.9599
Epoch 10/10
774/774 - 156s - loss: 0.1255 - accuracy: 0.9632 - val_loss: 0.1311 - 